---
# Optimal Region Selection for New Oil Well Development

This project aims to identify the most profitable and least risky region for developing new oil wells among three potential regions. Using advanced data analysis techniques and machine learning models, we assess each region's potential by analyzing geological data, predicting oil reserves, and evaluating financial metrics.
Our approach involves several key steps:

- Data Preparation: We begin by loading and inspecting the geological data from the three regions to understand its structure and ensure its readiness for analysis.
- Model Training and Validation: We train a Linear Regression model to predict the volume of oil reserves, using a split of training and validation data to ensure model accuracy. Performance metrics such as average predicted volume and Root Mean Squared Error (RMSE) are calculated for each region.
- Profit Calculation Preparation: Key financial parameters are established, and we calculate the minimum volume of reserves required to avoid losses. These values are compared with the average reserves in each region.
- Profit Analysis from Selected Wells: We select the top 200 wells with the highest predicted values in each region and calculate the total volume of reserves and expected profit.
- Risk and Profit Assessment: Employing the bootstrapping technique, we analyze the distribution of profits for each region, determining the average profit, confidence intervals, and the risk of losses.

The final recommendation is based on a comprehensive comparison of expected profits and associated risks.

---

###  Data Loading and Initial Inspection
#### Loading data from the three CSV files 

In [ ]:
# Import all necessary libraries
import pandas as pd
# import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import numpy as np
from scipy.stats import bootstrap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Load CSV files
data_region_0 = pd.read_csv('geo_data_0.csv')
data_region_1 = pd.read_csv('geo_data_1.csv')
data_region_2 = pd.read_csv('geo_data_2.csv')

# Inspecting first 5 rows of each dataset
display(data_region_0.head())
display(data_region_1.head())
display(data_region_2.head())

#### Inspecting each dataset

In [ ]:
# Inspecting the datasets
print(data_region_0.info())
print(data_region_1.info())
print(data_region_2.info())

# Statistical summaries of all datasets
display(data_region_0.describe())
display(data_region_1.describe())
display(data_region_2.describe())

# Check for missing values
display(data_region_0.isnull().sum())
display(data_region_1.isnull().sum())
display(data_region_2.isnull().sum())

#### Check For Duplicates

In [ ]:
# Check for completely duplicated rows in each dataframe
print(f"Region 0 duplicated rows: {data_region_0.duplicated().sum()}")
print(f"Region 1 duplicated rows: {data_region_1.duplicated().sum()}")
print(f"Region 2 duplicated rows: {data_region_2.duplicated().sum()}")

# Check for duplicates in each column of each dataframe
duplicates_region_0 = data_region_0.apply(lambda x: x.duplicated().sum())
duplicates_region_1 = data_region_1.apply(lambda x: x.duplicated().sum())
duplicates_region_2 = data_region_2.apply(lambda x: x.duplicated().sum())

print() 
print("Duplicates in Region 0:")
print(duplicates_region_0)
print("\nDuplicates in Region 1:")
print(duplicates_region_1)
print("\nDuplicates in Region 2:")
print(duplicates_region_2)

#### Removing Duplicates

In [ ]:
# removing duplicates in 'id' columns
data_region_0 = data_region_0.drop_duplicates(subset=['id'])
data_region_1 = data_region_1.drop_duplicates(subset=['id'])
data_region_2 = data_region_2.drop_duplicates(subset=['id'])

# Confirm that duplicates were removed
print(f"Region 0 duplicate ids after removal: {data_region_0['id'].duplicated().sum()}")
print(f"Region 1 duplicate ids after removal: {data_region_1['id'].duplicated().sum()}")
print(f"Region 2 duplicate ids after removal: {data_region_2['id'].duplicated().sum()}")

#### Analyzing Distributions in Each Dataframe Through Charts

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Distribution of Oil Reserves in Each Region', fontsize=18, fontweight='bold')

axes[0].hist(data_region_0['product'], bins=50, color='blue')
axes[0].set_title('Region 0')

axes[1].hist(data_region_1['product'], bins=50, color='green')
axes[1].set_title('Region 1')

axes[2].hist(data_region_2['product'], bins=50, color='red')
axes[2].set_title('Region 2')

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

#### Graph Analysis: Distribution of Oil Reserves by Region
**Region 0:**
- The distribution of oil reserves shows a relatively uniform pattern with multiple peaks.
- There are frequent values around 50, 75, 100, 125, and 150 thousand barrels.
- The number of oil wells with these reserve levels ranges from about 500 to over 3000.

**Region 1:**
- The distribution of oil reserves in Region 1 exhibits a highly periodic pattern with distinct peaks at regular intervals.
- Oil reserves are concentrated around 0, 30, 60, 90, 120, and 140 thousand barrels.
- The number of oil wells for these reserve levels is consistently around 8000.
- This periodic pattern suggests that specific reserve levels are significantly more common.

**Region 2:**
- The distribution of oil reserves in Region 2 shows a more complex pattern with multiple peaks.
- Frequent values are observed around 25, 50, 75, 100, 125, and 150 thousand barrels.
- The number of oil wells ranges from about 500 to over 3000, similar to Region 0.

**Uniformity and Variability:** Regions 0 and 2 exhibit a more variable distribution with multiple peaks, indicating variability in oil reserves. Region 1 shows a highly regular and periodic distribution, suggesting specific reserve levels are more common.

---
### Train and Test the Model for Each Region
- Data Splitting: Each dataset was split into training and validation sets with a ratio of 75:25.
- Model Training: A Linear Regression model was trained on the training set.
- Predictions: Predictions were made on the validation set.
- Performance Metrics: The average predicted volume and Root Mean Squared Error (RMSE) were calculated for each region

#### Split and Scale the Data

In [ ]:
# The target column is 'product'
TARGET_COLUMN = 'product'

# Define a function to split and scale data
def split_and_scale_data(data, target_column):
    features = data.drop(target_column, axis=1).drop('id', axis=1)
    target = data[target_column]
    X_train, X_valid, y_train, y_valid = train_test_split(features, target, test_size=0.25, random_state=42)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)
    
    return X_train_scaled, X_valid_scaled, y_train, y_valid

# Split and scale the datasets
X_train_0, X_valid_0, y_train_0, y_valid_0 = split_and_scale_data(data_region_0, TARGET_COLUMN)
X_train_1, X_valid_1, y_train_1, y_valid_1 = split_and_scale_data(data_region_1, TARGET_COLUMN)
X_train_2, X_valid_2, y_train_2, y_valid_2 = split_and_scale_data(data_region_2, TARGET_COLUMN)

#### Train the Model and Make Predictions

In [ ]:
# Define a function to train and predict
def train_and_predict(X_train, y_train, X_valid):
    model = LinearRegression()
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    return predictions, model

# Train and predict for each region
predictions_0, model_0 = train_and_predict(X_train_0, y_train_0, X_valid_0)
predictions_1, model_1 = train_and_predict(X_train_1, y_train_1, X_valid_1)
predictions_2, model_2 = train_and_predict(X_train_2, y_train_2, X_valid_2)

#### Save the Predictions and Correct Answers

In [ ]:
# Save predictions and actual values
results_0 = pd.DataFrame({'actual': y_valid_0, 'predicted': predictions_0})
results_1 = pd.DataFrame({'actual': y_valid_1, 'predicted': predictions_1})
results_2 = pd.DataFrame({'actual': y_valid_2, 'predicted': predictions_2})

display(results_0.head())
display(results_1.head())
display(results_2.head())

#### Print the Average Volume of Predicted Reserves and Model RMSE

In [ ]:
# Calculate and print average volume and RMSE for each region
def calculate_rmse(actual, predicted):
    return np.sqrt(mean_squared_error(actual, predicted))

avg_predicted_volumes = []
rmses = []

for i, (y_valid, predictions) in enumerate([(y_valid_0, predictions_0), (y_valid_1, predictions_1), (y_valid_2, predictions_2)]):
    avg_predicted = predictions.mean()
    rmse = calculate_rmse(y_valid, predictions)
    
    avg_predicted_volumes.append(avg_predicted)
    rmses.append(rmse)
    
    print(f"Region {i} - Avg Predicted Volume: {avg_predicted:.2f}, RMSE: {rmse:.2f}")

#### Interpretation of RMSE Results by Region:

**Region 0 :**
- The RMSE of 37.69 indicates the model's predictions have a relatively large spread around the actual values. This suggests the model has moderate accuracy, but there is still significant variance between predicted and actual values.

**Region 1:**
- The RMSE of 0.89 is very low, indicating that the model's predictions are very close to the actual values. This high accuracy suggests that the model fits the data for Region 1 very well.

**Region 2:**
- The RMSE of 40.08 indicates a relatively large spread around the actual values, similar to Region 0. This suggests that the model has moderate accuracy with significant variance between predicted and actual values.

#### General RMSE Observations:

**Model Performance:**
- The model performs exceptionally well in Region 1 with a very low RMSE, suggesting a strong fit to the data.
- The model's performance in Regions 0 and 2 is less accurate, with higher RMSE values indicating greater variance in predictions.

**Data Characteristics:**
- The low RMSE in Region 1 may be attributed to the distinct, discrete nature of the data, as seen in the histogram. This distinct pattern may be easier for the model to learn and predict accurately.
- Regions 0 and 2 have more continuous distributions with multiple peaks, which might introduce complexity in the data, making it harder for the model to predict accurately.

#### Prepare for profit calculation:
- Calculate the sufficient volume of reserves to avoid losses.
- Compare this with the average volume of reserves in each region.

In [ ]:
# Key values for profit calculation
PRICE_PER_UNIT = 4.5 * 1000  # $4500 per thousand barrels
COST_PER_WELL = 100000000 / 200  # $500000 per well
TOP_N_WELLS = 200

# Combine the datasets into a list 
data_regions = [data_region_0, data_region_1, data_region_2]

# Calculate the volume of reserves sufficient for developing a new well without losses
volume_sufficient = COST_PER_WELL / PRICE_PER_UNIT
print(f"Volume of reserves sufficient for developing a new well without losses: {volume_sufficient:.2f}")

# Compare with the average volume of reserves in each region
for i, data_region in enumerate(data_regions):
    avg_volume = data_region['product'].mean()
    print(f"Region {i} - Avg Volume of Reserves: {avg_volume:.2f}")

#### Summary and Interpretaton of Potential Profit and Risks:

**Sufficient Volume to Avoid Losses: 111.11 thousand barrels.** This means that any well producing less than this volume will incur a loss.

**Average Volume of Reserves:**

- Region 0: The average volume of reserves is 92.50 thousand barrels, which is below the sufficient volume to avoid losses.
- Region 1: The average volume of reserves is 68.82 thousand barrels, significantly below the sufficient volume to avoid losses.
- Region 2: The average volume of reserves is 95.00 thousand barrels, also below the sufficient volume to avoid losses.

**Implications:**
- All three regions have average reserve volumes below the threshold needed to avoid losses. This suggests that, on average, wells in these regions are not profitable based on the current cost and revenue structure.
- Region 1: Has the lowest average volume of reserves, making it the least likely to meet the profitability threshold.
- Regions 0 and 2: Have closer average volumes to the sufficient volume but are still below the threshold, indicating potential risk in profitability.

---
### Calculate Profit from Selected Wells
- Select the top 200 wells based on model predictions.
- Calculate the total volume and profit for these wells.
- Suggest the best region based on expected profit.

#### Define the Profit Function 

In [ ]:
# Function to calculate profit from selected oil wells
def calculate_profit_from_wells(data, top_n, price_per_unit=PRICE_PER_UNIT, cost_per_well=COST_PER_WELL):
    top_wells = data.nlargest(top_n, 'predicted')
    
    # Summarize the target volume of reserves
    total_volume = top_wells['actual'].sum()
    
    # Calculate profit
    revenue = total_volume * price_per_unit
    cost = top_n * cost_per_well
    profit = revenue - cost
    return total_volume, profit

#### Calculate Profit for Each Region

In [ ]:
# Calculate profit for each region
profits = []
total_volumes = []

for i, result in enumerate([results_0, results_1, results_2]):
    total_volume, profit = calculate_profit_from_wells(result, TOP_N_WELLS)
    total_volumes.append(total_volume)
    profits.append(profit)
    print(f"Region {i} - Total Volume: {total_volume:.2f}, Profit: ${profit:.2f}")

# Provide findings and suggest a region
best_region = np.argmax(profits)
print(f"\nThe best region for oil wells' development is Region {best_region} with an expected profit of ${profits[best_region]:.2f}.")

---
### Calculate Risks and Profit for Each Region Using Bootstrapping:

#### Use the bootstrapping technique with 1000 samples to find the distribution of profit

In [ ]:
# Bootstrapping technique to calculate profits 
def bootstrap_profit(data, n_samples=1000, sample_size=500, top_n=TOP_N_WELLS, price_per_unit=PRICE_PER_UNIT, cost_per_well=COST_PER_WELL, random_seed=42):
    np.random.seed(random_seed)  # Set the random seed
    profits = []
    for _ in range(n_samples):
        sample = data.sample(n=sample_size, replace=True)
        _, profit = calculate_profit_from_wells(sample, top_n, price_per_unit, cost_per_well)
        profits.append(profit)
    return np.array(profits)

#### Find average profit, 95% confidence interval and risk of losses:

In [ ]:
# Function to calculate average profit sats
def calculate_statistics(profits, confidence_level=0.95):
    avg_profit = np.mean(profits)
    lower_percentile = (1 - confidence_level) / 2 * 100
    upper_percentile = (1 + confidence_level) / 2 * 100
    conf_interval = np.percentile(profits, [lower_percentile, upper_percentile])
    risk_of_loss = (profits < 0).mean() * 100  
    return avg_profit, conf_interval, risk_of_loss

# Process each region
results = []
bootstrap_profits = []

for data_region in data_regions:
    # Split and scale data
    X_train, X_valid, y_train, y_valid = split_and_scale_data(data_region, 'product')
    # Train and predict
    predictions, model = train_and_predict(X_train, y_train, X_valid)
    # Save predictions and actual values
    result = pd.DataFrame({'actual': y_valid, 'predicted': predictions})
    results.append(result)
    # Calculate profits using bootstrapping
    profits = bootstrap_profit(result)  
    bootstrap_profits.append(profits)

# Calculate statistics and financial risks for each region and print the results
region_stats = []
for i, profits in enumerate(bootstrap_profits):
    avg_profit, conf_interval, risk_of_loss = calculate_statistics(profits)
    region_stats.append((avg_profit, conf_interval, risk_of_loss))
    print(f"Region {i} - Average Profit: ${avg_profit:.2f}, 95% Confidence Interval: (${conf_interval[0]:.2f}, ${conf_interval[1]:.2f}), Risk of Loss: {risk_of_loss:.2f}%")

#### Provide findings: suggest a region for development of oil wells and justify the choice:

In [ ]:
# Filter regions with risk of loss lower than 2.5%
filtered_region_stats = [(i, stats) for i, stats in enumerate(region_stats) if stats[2] < 2.5]


# Provide findings and suggest a region
#best_region = np.argmax([stat[0] for stat in region_stats])
#print(f"\nThe best region for oil wells' development is Region {best_region} with an expected profit of ${region_stats[best_region][0]:.2f}.")

# Suggest the best region for development
if filtered_region_stats:
    best_region = max(filtered_region_stats, key=lambda x: x[1][0])
    print(f"\nThe best region for oil wells' development is Region {best_region[0]} with an expected profit of ${best_region[1][0]:.2f}.")
else:
    print("\nNo region meets the risk of loss criteria.")  

#### Suggesting a Region for Development:

**Region 1** is suggested for the development of oil wells based on the following results:

- Risk of Loss: Region 1 has a risk of loss of 1.60%, which is the only region below the threshold of 2.5%.
- Profit: While Region 1 doesn't have the highest average profit, its risk of loss is lower compared to other regions, making it a safer investment according to the criteria.

#### Plot Profit Distributions

In [ ]:
# Plot profit distributions for each region
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=False)
fig.suptitle('Profit Distributions for Each Region (in millions USD)', fontsize=18, fontweight='bold')

# Scale profits to millions
bootstrap_profits_millions = [profits / 1e6 for profits in bootstrap_profits]

# Define colors
colors = ['blue', 'green', 'red']

# Titles for each region
titles = ['Region 0', 'Region 1', 'Region 2']

# Plot each regions profit distribution
for i, ax in enumerate(axes):
    ax.hist(bootstrap_profits_millions[i], bins=50, color=colors[i], alpha=0.7, edgecolor='black')
    ax.set_title(titles[i], fontsize=14)
    ax.set_xlabel('Profit (Millions USD)', fontsize=12)
    ax.set_ylabel('Frequency (Number of Bootstrap Samples)', fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.6)

# Adjust layout for better fit
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

#### Graph Analysis: Profit Distributions by Region (Bootsrap Sampling)
- The histograms show that Region 1 has a lower frequency of negative profits compared to Regions 0 and 2, suggesting a relatively lower risk of loss. This aligns with the calculated risk of loss at 1.60%, which is the only risk of loss calculated below the threshold.
- Regions 0 and 2 show significant variability in profit outcomes with wider distributions and substantial risks of negative profits.

---
# Conclusions and Final Recommendations:

**Sufficient Volume to Avoid Losses:**
- Threshold: 111.11 thousand barrels. Any well producing less than this volume will incur a loss.

#### Total Volume and Profit Analysis
**Region 0:** 
- Total Volume: 29,930.07 barrels
- Profit: &dollar;34,685,297.87 
- Average Profit: &dollar;4,035,720.64 
- 95% Confidence Interval: -\\$1,376,900.03 - \\$9,028,483.11
- Risk of Loss: 5.90%

**Region 1:**
- Total Volume: 27,589.08 barrels
- Profit: &dollar;24,150,866.97
- Average Profit: &dollar;4,302,083.52
- 95% Confidence Interval: \\$430,525.20 - \\$8,473,134.67
- Risk of Loss: 1.60%

**Region 2:**
- Total Volume: 27,548.03 barrels
- Profit: &dollar;23,966,130.01
- Average Profit: &dollar;3,842,144.29
- 95% Confidence Interval: -\\$1,275,898.77 - \\$9,101,344.40
- Risk of Loss: 8.00%

#### Final Recommendations: 
**Region Selection:**
- Based on the bootstrap samples, Region 1 has the lowest risk of loss at 1.30%, which is below the threshold of 2.5%. This makes it a safer investment compared to Regions 0 and 2, both of which have higher risks of loss at 5.90% and 8.00%, respectively.
- While Region 1 does not have the highest average profit, it is the safest investment due to its significantly lower risk of loss compared to the other regions.
- Therefore, **Region 1** is recommended for oil well development based on the combination of its lower risk and consistent profitability. This ensures a strategic balance between minimizing risk and achieving reliable returns.
---